## Day 3 - Engine schematics

Part 1: Numbers adjacent (inc. diag) are part numbers. Sum them all!

In [1]:
with open("./example.txt") as f:
    example_lines = []
    for line in f.readlines():
        example_lines.append(line.strip())

with open("./input.txt") as f:
    input_lines = []
    for line in f.readlines():
        input_lines.append(line.strip())

In [2]:
example_lines

['467..114..',
 '...*......',
 '..35..633.',
 '......#...',
 '617*......',
 '.....+.58.',
 '..592.....',
 '......755.',
 '...$.*....',
 '.664.598..']

Let's use the fact lines are all equal length.

In [50]:
def check_number_adjacency(
        line_grid: list[list[str]],
        number_start_coords: tuple[int, int],
        number_end_coords: tuple[int, int],
        i_max: int,
        j_max: int,
    ) -> bool:
    i_start, j_start = number_start_coords
    i_end, j_end = number_end_coords

    assert i_start == i_end, f"Why is i_start and i_end different? {i_start, i_end}"


    # e.g.
    # start(2,2) end(2,5)
    #  > watch out for going over 'edges'
    # have to check above (1,1) (1,2) (1,3) (1, 4) (1,5) (1,6)
    # have to check sides (2,1) (2,4)
    # have to check below (3,1) (3,2) (3,3) (3,4) (3,5) (3,6)

    coords_to_check = []
    # ABOVE
    if (i_row_above := i_start-1) >= 0:
        for j in range(j_start-1, j_end+2):
            if j >= 0 and j<=j_max:
                coords_to_check.append(
                    (i_row_above, j)
                )
    # SIDES
    if (j_left_side := j_start - 1) >= 0:
        coords_to_check.append(
            (i_start, j_left_side)
        )
    if (j_right_side := j_end + 1) <= j_max:
        coords_to_check.append(
            (i_end, j_right_side)
        )
    # BELOW
    if (i_row_below := i_start + 1) <= i_max:
        for j in range(j_start-1, j_end+2):
            if j >= 0 and j<=j_max:
                coords_to_check.append(
                    (i_row_below, j)
                )
    
    legitimate = False

    for coords in coords_to_check:
        coord_i, coord_j = coords
        item = line_grid[coord_i][coord_j]
        try:
            int(item)
        except:
            if item != ".":
                legitimate = True
                break
    
    return legitimate

In [52]:
def part1(lines: list[str]) -> int:
    line_grid = [[*line] for line in lines]
    # all lines equal length
    X_length = len(line_grid[0])
    Y_length = len(line_grid)

    # line_grid[i][j]

    building_number = False
    number_str = ""
    number_start_coords = (None, None)
    number_end_coords = (None, None)

    legit_numbers = []

    for i in range(Y_length):
        for j in range(X_length):
            # Such that i,j is the ith row and jth col
            if building_number:
                try:
                    number_str += str(int(line_grid[i][j]))
                    number_end_coords = (i,j)
                    if j == X_length:
                        # If we've reached the edge with a legit number, then building number complete!
                        legit = check_number_adjacency(
                            line_grid=line_grid,
                            number_start_coords=number_start_coords,
                            number_end_coords=number_end_coords,
                            i_max=Y_length-1,
                            j_max=X_length-1,
                        )
                        if legit:
                            legit_numbers.append(int(number_str))
                            building_number = False
                            number_str = ""
                            number_start_coords = (None, None)
                            number_end_coords = (None, None)
                        else:
                            building_number = False
                            number_str = ""
                            number_start_coords = (None, None)
                            number_end_coords = (None, None)
                except:
                    # Have completed building the number now
                    legit = check_number_adjacency(
                        line_grid=line_grid,
                        number_start_coords=number_start_coords,
                        number_end_coords=number_end_coords,
                        i_max=Y_length-1,
                        j_max=X_length-1,
                    )
                    if legit:
                        legit_numbers.append(int(number_str))
                        building_number = False
                        number_str = ""
                        number_start_coords = (None, None)
                        number_end_coords = (None, None)
                    else:
                        building_number = False
                        number_str = ""
                        number_start_coords = (None, None)
                        number_end_coords = (None, None)              
            else:
                try:
                    number_str += str(int(line_grid[i][j]))
                    building_number = True
                    number_start_coords = (i,j)
                    number_end_coords = (i,j)
                except:
                    continue

    return sum(legit_numbers)

    

assert part1(example_lines) == 4361
part1(input_lines)

532445

Part 2: Gear ratios. Two numbers are a gear ratio if both adjacent to same "*", multiply them to get value, sum all values

plan, since it's not huge let's just do a full grid search of all combos lol.

In [53]:
def gear_ratio(
        line_grid: list[list[str]],
        number_start_coords_A: tuple[int, int],
        number_end_coords_A: tuple[int, int],
        number_start_coords_B: tuple[int, int],
        number_end_coords_B: tuple[int, int],
        i_max: int,
        j_max: int,
    ) -> bool:
    i_start_A, j_start_A = number_start_coords_A
    i_end_A, j_end_A = number_end_coords_A

    i_start_B, j_start_B = number_start_coords_B
    i_end_B, j_end_B = number_end_coords_B

    assert i_start_A == i_end_A, f"Why is i_start and i_end different for A? {i_start_A, i_end_A}"
    assert i_start_B == i_end_B, f"Why is i_start and i_end different for A? {i_start_B, i_end_B}"

    # RECALL LOGIC for what is adjacent
    # start(2,2) end(2,5)
    #  > watch out for going over 'edges'
    # have to check above (1,1) (1,2) (1,3) (1, 4) (1,5) (1,6)
    # have to check sides (2,1) (2,4)
    # have to check below (3,1) (3,2) (3,3) (3,4) (3,5) (3,6)

    coords_to_check_A = []

    # **************** DO IT FOR A **************

    # ABOVE
    if (i_row_above := i_start_A-1) >= 0:
        for j in range(j_start_A-1, j_end_A+2):
            if j >= 0 and j<=j_max:
                coords_to_check_A.append(
                    (i_row_above, j)
                )
    # SIDES
    if (j_left_side := j_start_A - 1) >= 0:
        coords_to_check_A.append(
            (i_start_A, j_left_side)
        )
    if (j_right_side := j_end_A + 1) <= j_max:
        coords_to_check_A.append(
            (i_end_A, j_right_side)
        )
    # BELOW
    if (i_row_below := i_start_A + 1) <= i_max:
        for j in range(j_start_A-1, j_end_A+2):
            if j >= 0 and j<=j_max:
                coords_to_check_A.append(
                    (i_row_below, j)
                )
    
    # ************ DO SAME FOR B ************

    coords_to_check_B = []

    # ABOVE
    if (i_row_above := i_start_B-1) >= 0:
        for j in range(j_start_B-1, j_end_B+2):
            if j >= 0 and j<=j_max:
                coords_to_check_B.append(
                    (i_row_above, j)
                )
    # SIDES
    if (j_left_side := j_start_B - 1) >= 0:
        coords_to_check_B.append(
            (i_start_B, j_left_side)
        )
    if (j_right_side := j_end_B + 1) <= j_max:
        coords_to_check_B.append(
            (i_end_B, j_right_side)
        )
    # BELOW
    if (i_row_below := i_start_B + 1) <= i_max:
        for j in range(j_start_B-1, j_end_B+2):
            if j >= 0 and j<=j_max:
                coords_to_check_B.append(
                    (i_row_below, j)
                )

    # FIND * COORDS FOR BOTH A AND B
    # RETURN TRUE IF THEY SHARE THIS COORD

    star_coords_A = []

    for coords in coords_to_check_A:
        coord_i, coord_j = coords
        item = line_grid[coord_i][coord_j]
        if item == "*":
            star_coords_A.append(coords)
    
    star_coords_B = []

    for coords in coords_to_check_B:
        coord_i, coord_j = coords
        item = line_grid[coord_i][coord_j]
        if item == "*":
            star_coords_B.append(coords)

    crossover = set(star_coords_A).intersection(set(star_coords_B))
    crossover_coords_found = False
    if len(crossover) > 0:
        if len(crossover) == 1:
            crossover_coords_found = True
        else:
            raise ValueError("What's going on...")
      
    return crossover_coords_found

In [56]:
def get_all_number_dicts(lines: list[str]) -> list[dict]:
    line_grid = [[*line] for line in lines]
    # all lines equal length
    X_length = len(line_grid[0])
    Y_length = len(line_grid)

    # line_grid[i][j]

    building_number = False
    number_str = ""
    number_start_coords = (None, None)
    number_end_coords = (None, None)

    all_legit_number_coords = [] # each entry is a dictionary with stucture:
            # e.g. {start: (int,int), end: (int,int), value: int}

    for i in range(Y_length):
        for j in range(X_length):
            # Such that i,j is the ith row and jth col
            if building_number:
                try:
                    number_str += str(int(line_grid[i][j]))
                    number_end_coords = (i,j)
                    if j == X_length:
                        # If we've reached the edge with a legit number, then building number complete!
                        legit = check_number_adjacency(
                            line_grid=line_grid,
                            number_start_coords=number_start_coords,
                            number_end_coords=number_end_coords,
                            i_max=Y_length-1,
                            j_max=X_length-1,
                        )
                        if legit:
                            all_legit_number_coords.append(
                                {
                                    "start": number_start_coords,
                                    "end": number_end_coords,
                                    "value": int(number_str),
                                })
                            building_number = False
                            number_str = ""
                            number_start_coords = (None, None)
                            number_end_coords = (None, None)
                        else:
                            building_number = False
                            number_str = ""
                            number_start_coords = (None, None)
                            number_end_coords = (None, None)
                except:
                    # Have completed building the number now
                    legit = check_number_adjacency(
                        line_grid=line_grid,
                        number_start_coords=number_start_coords,
                        number_end_coords=number_end_coords,
                        i_max=Y_length-1,
                        j_max=X_length-1,
                    )
                    if legit:
                        all_legit_number_coords.append(
                                {
                                    "start": number_start_coords,
                                    "end": number_end_coords,
                                    "value": int(number_str),
                                })
                        building_number = False
                        number_str = ""
                        number_start_coords = (None, None)
                        number_end_coords = (None, None)
                    else:
                        building_number = False
                        number_str = ""
                        number_start_coords = (None, None)
                        number_end_coords = (None, None)              
            else:
                try:
                    number_str += str(int(line_grid[i][j]))
                    building_number = True
                    number_start_coords = (i,j)
                    number_end_coords = (i,j)
                except:
                    continue

    return all_legit_number_coords

example_number_dicts = get_all_number_dicts(example_lines)
input_number_dicts = get_all_number_dicts(input_lines)

example_number_dicts

[{'start': (0, 0), 'end': (0, 2), 'value': 467},
 {'start': (2, 2), 'end': (2, 3), 'value': 35},
 {'start': (2, 6), 'end': (2, 8), 'value': 633},
 {'start': (4, 0), 'end': (4, 2), 'value': 617},
 {'start': (6, 2), 'end': (6, 4), 'value': 592},
 {'start': (7, 6), 'end': (7, 8), 'value': 755},
 {'start': (9, 1), 'end': (9, 3), 'value': 664},
 {'start': (9, 5), 'end': (9, 7), 'value': 598}]

In [61]:
import itertools

def part2(
        lines: list[str],
        number_dicts: list[dict]
    ) -> int:
    line_grid = [[*line] for line in lines]
    # all lines equal length
    j_max = len(line_grid[0]) - 1
    i_max = len(line_grid) - 1

    ratio_sum = 0

    for pair in itertools.combinations(number_dicts, r=2):
        dict_A, dict_B = pair
        if gear_ratio(
            line_grid,
            dict_A["start"],
            dict_A["end"],
            dict_B["start"],
            dict_B["end"],
            i_max,
            j_max
            ):
            ratio_sum += dict_A["value"] * dict_B["value"]
    
    return ratio_sum

assert part2(example_lines, example_number_dicts) == 467835, f"rip {part2(example_lines, example_number_dicts)}"

part2(input_lines, input_number_dicts)


79842967